In [1]:
image_data = []
labels = []

label_dict = {
    "Aerodactyl": 0,
    "Bulbasaur": 1,
    "Charmander": 2,
    "Dratini": 3,
    "Fearow": 4,
    "Meowth": 5,
    "Pikachu": 6,
    "Psyduck": 7,
    "Spearow": 8,
    "Squirtle": 9,
}

In [2]:
import os
folders = os.listdir("Train")
print(folders)

['Aerodactyl', 'Bulbasaur', 'Charmander', 'Dratini', 'Fearow', 'Meowth', 'Pikachu', 'Psyduck', 'Spearow', 'Squirtle']


In [3]:
from keras.preprocessing import image

In [4]:
for ix in folders:
  path = os.path.join("Train", ix)
  for im in os.listdir(path):
    img = image.load_img(os.path.join(path,im), target_size=(224, 224))
    img_array = image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix])

c:\users\linta.kumar\appdata\local\programs\python\python37\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [5]:
print(len(image_data))

1645


In [6]:
print(len(labels))

1645


In [7]:
import random
combined = list(zip(image_data, labels))
random.shuffle(combined)

In [8]:
image_data[:], labels[:] = zip(*combined)

In [9]:
print(len(image_data))

1645


In [10]:
import numpy as np

In [11]:
X_train = np.array(image_data)
Y_train = np.array(labels)

print(X_train.shape, Y_train.shape)

(1645, 224, 224, 3) (1645,)


In [12]:
from keras.utils import to_categorical

In [13]:
Y_train = to_categorical(Y_train)

In [14]:
Y_train[:10]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [15]:
X = X_train.reshape((-1,224,224,1))

In [16]:
print(X.shape, Y_train.shape)

(4935, 224, 224, 1) (1645, 10)


In [17]:
X_train = X_train/255

In [18]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import np_utils
import tensorflow
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam

In [21]:
vggmodel = VGG16(weights='imagenet', include_top= False, input_shape=(224,224, 3))

In [22]:
for layer in vggmodel.layers:
    layer.trainable = False

In [23]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [110]:
av1 = GlobalAveragePooling2D()(vggmodel.output)
fc1 = Dense(256, activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(10, activation='softmax')(d1)

In [111]:
model_new = Model(inputs = vggmodel.input , outputs=fc2)
model_new.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)     

In [112]:
adam = Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [113]:
feature_extractor = model_new.predict(X_train)

In [114]:
features = feature_extractor.reshape(feature_extractor.shape[0],-1)

In [115]:
X_for_RF = features

In [116]:
from sklearn.ensemble import RandomForestClassifier

In [117]:
rfmodel = RandomForestClassifier(n_estimators = 50, random_state = 2)

In [118]:
rfmodel.fit(X_for_RF, Y_train)

RandomForestClassifier(n_estimators=50, random_state=2)

In [119]:
rfmodel.score(X_for_RF, Y_train)

0.9896656534954408

In [120]:
# Prediction on Test Set

In [121]:
folders = os.listdir("Test/images")
print(folders)

['test_0.jpg', 'test_1.jpg', 'test_10.jpg', 'test_100.jpg', 'test_101.jpg', 'test_102.jpg', 'test_103.jpg', 'test_104.jpg', 'test_105.jpg', 'test_106.jpg', 'test_107.jpg', 'test_108.jpg', 'test_109.jpg', 'test_11.jpg', 'test_110.jpg', 'test_111.jpg', 'test_112.jpg', 'test_113.jpg', 'test_114.jpg', 'test_115.jpg', 'test_116.jpg', 'test_117.jpg', 'test_118.jpg', 'test_119.jpg', 'test_12.jpg', 'test_120.jpg', 'test_121.jpg', 'test_122.jpg', 'test_123.jpg', 'test_124.jpg', 'test_125.jpg', 'test_126.jpg', 'test_127.jpg', 'test_128.jpg', 'test_129.jpg', 'test_13.jpg', 'test_130.jpg', 'test_131.jpg', 'test_132.jpg', 'test_133.jpg', 'test_134.jpg', 'test_135.jpg', 'test_136.jpg', 'test_137.jpg', 'test_138.jpg', 'test_139.jpg', 'test_14.jpg', 'test_140.jpg', 'test_141.jpg', 'test_142.jpg', 'test_143.jpg', 'test_144.jpg', 'test_145.jpg', 'test_146.jpg', 'test_147.jpg', 'test_148.jpg', 'test_149.jpg', 'test_15.jpg', 'test_150.jpg', 'test_151.jpg', 'test_152.jpg', 'test_153.jpg', 'test_154.jpg', '

In [122]:
from pathlib import Path
py = Path("Test/images")

In [123]:
X_test = np.array(image_datay)

In [124]:

X_test = X_test/255

In [125]:
testfeature_extractor = model_new.predict(X_test)
testfeatures = testfeature_extractor.reshape(testfeature_extractor.shape[0],-1)

In [126]:
predictions = rfmodel.predict(testfeatures)

In [127]:
predictions

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [48]:
imagenames

['test_0.jpg',
 'test_1.jpg',
 'test_10.jpg',
 'test_100.jpg',
 'test_101.jpg',
 'test_102.jpg',
 'test_103.jpg',
 'test_104.jpg',
 'test_105.jpg',
 'test_106.jpg',
 'test_107.jpg',
 'test_108.jpg',
 'test_109.jpg',
 'test_11.jpg',
 'test_110.jpg',
 'test_111.jpg',
 'test_112.jpg',
 'test_113.jpg',
 'test_114.jpg',
 'test_115.jpg',
 'test_116.jpg',
 'test_117.jpg',
 'test_118.jpg',
 'test_119.jpg',
 'test_12.jpg',
 'test_120.jpg',
 'test_121.jpg',
 'test_122.jpg',
 'test_123.jpg',
 'test_124.jpg',
 'test_125.jpg',
 'test_126.jpg',
 'test_127.jpg',
 'test_128.jpg',
 'test_129.jpg',
 'test_13.jpg',
 'test_130.jpg',
 'test_131.jpg',
 'test_132.jpg',
 'test_133.jpg',
 'test_134.jpg',
 'test_135.jpg',
 'test_136.jpg',
 'test_137.jpg',
 'test_138.jpg',
 'test_139.jpg',
 'test_14.jpg',
 'test_140.jpg',
 'test_141.jpg',
 'test_142.jpg',
 'test_143.jpg',
 'test_144.jpg',
 'test_145.jpg',
 'test_146.jpg',
 'test_147.jpg',
 'test_148.jpg',
 'test_149.jpg',
 'test_15.jpg',
 'test_150.jpg',
 'test_

In [129]:
Y_test = np.argmax(predictions, axis=1)

In [130]:
np.argmax(predictions, axis=1)

array([0, 0, 1, 2, 0, 0, 0, 0, 6, 0, 6, 0, 6, 0, 0, 6, 6, 6, 6, 6, 6, 0,
       0, 0, 1, 6, 0, 6, 0, 6, 0, 0, 0, 6, 0, 1, 7, 7, 7, 0, 0, 0, 0, 7,
       0, 7, 1, 0, 0, 9, 0, 0, 0, 8, 8, 8, 8, 1, 8, 8, 8, 8, 8, 8, 0, 8,
       0, 9, 1, 0, 9, 1, 0, 0, 9, 0, 9, 9, 9, 1, 0, 9, 9, 0, 0, 0, 0, 9,
       9, 9, 2, 0, 9, 2, 1, 2, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 6, 2, 2,
       0, 0, 2, 0, 0, 0, 2, 9, 0, 2, 0, 0, 2, 9, 6, 0, 1, 2, 2, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 3, 0, 3, 0, 6, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       0, 4, 4, 4, 0, 0, 9, 0, 0, 6, 0], dtype=int64)

In [131]:
Y_test[:10]

array([0, 0, 1, 2, 0, 0, 0, 0, 6, 0], dtype=int64)

In [132]:
Y_test.shape

(187,)

In [138]:
a = np.array(imagenames)
b = Y_test
a = a.reshape(a.shape[0],-1)
a.shape
b = b.reshape(b.shape[0],-1)

In [139]:
result = np.zeros((Y_test.shape[0],2))

In [140]:
result = np.hstack((a,b))

In [141]:
import pandas as pd

In [142]:
df = pd.DataFrame(data=result, columns=["Name","Class"])
df.head()
df.to_csv('test.csv')

In [143]:
df.head()

,Name,Class
0,test_0.jpg,0
1,test_1.jpg,0
2,test_10.jpg,1
3,test_100.jpg,2
4,test_101.jpg,0


In [144]:
YTest = pd.read_csv("Test/sample_submission.csv")
YTest.head()
YTest.drop(columns=['Class'])

,Name
0,test_32.jpg
1,test_147.jpg
2,test_171.jpg
3,test_114.jpg
4,test_162.jpg
...,...
182,test_150.jpg
183,test_72.jpg
184,test_95.jpg
185,test_88.jpg


In [145]:
Yd = pd.merge(df, YTest, how='right', left_on='Name', right_on='Name')

In [146]:
Yd.head()

,Name,Class_x,Class_y
0,test_32.jpg,0,0
1,test_147.jpg,8,0
2,test_171.jpg,9,0
3,test_114.jpg,6,0
4,test_162.jpg,1,0


In [147]:
Yd.to_csv('ans.csv')